In [2]:
import os
import csv
import sys
import random
import scipy
import numpy as np
import xgboost as xgb
# read in data


workspace_path = '/home/irashadow/python_workspace/Kaggle_Comp/Santander/'
month_dict_inv = dict()
month_dict_inv[0] = '2015-01'
month_dict_inv[1] = '2015-02'
month_dict_inv[2] = '2015-03'
month_dict_inv[3] = '2015-04'
month_dict_inv[4] = '2015-05'
month_dict_inv[5] = '2015-06'
month_dict_inv[6] = '2015-07'
month_dict_inv[7] = '2015-08'
month_dict_inv[8] = '2015-09'
month_dict_inv[9] = '2015-10'
month_dict_inv[10] = '2015-11'
month_dict_inv[11] = '2015-12'
month_dict_inv[12] = '2016-01'
month_dict_inv[13] = '2016-02'
month_dict_inv[14] = '2016-03'
month_dict_inv[15] = '2016-04'
month_dict_inv[16] = '2016-05'
month_dict_inv[17] = '2016-06'
label_dict = dict()
label_dict[1] = 'ind_ahor_fin_ult1'
label_dict[2] = 'ind_aval_fin_ult1'
label_dict[3] = 'ind_cco_fin_ult1'
label_dict[4] = 'ind_cder_fin_ult1'
label_dict[5] = 'ind_cno_fin_ult1'
label_dict[6] = 'ind_ctju_fin_ult1'
label_dict[7] = 'ind_ctma_fin_ult1'
label_dict[8] = 'ind_ctop_fin_ult1'
label_dict[9] = 'ind_ctpp_fin_ult1'
label_dict[10] = 'ind_deco_fin_ult1'
label_dict[11] = 'ind_deme_fin_ult1'
label_dict[12] = 'ind_dela_fin_ult1'
label_dict[13] = 'ind_ecue_fin_ult1'
label_dict[14] = 'ind_fond_fin_ult1'
label_dict[15] = 'ind_hip_fin_ult1'
label_dict[16] = 'ind_plan_fin_ult1'
label_dict[17] = 'ind_pres_fin_ult1'
label_dict[18] = 'ind_reca_fin_ult1'
label_dict[19] = 'ind_tjcr_fin_ult1'
label_dict[20] = 'ind_valo_fin_ult1'
label_dict[21] = 'ind_viv_fin_ult1'
label_dict[22] = 'ind_nomina_ult1'
label_dict[23] = 'ind_nom_pens_ult1'
label_dict[24] = 'ind_recibo_ult1'

In [3]:
test_data_path = './test/5_4_cust_date_prod_L4_standard.test'
#test_data_path = './test/5_5_cust_date_prod_L4_expand.test'

dtest = xgb.DMatrix(test_data_path)

In [30]:
cust_2_labels_dict = dict()

pos_case_dict = dict()
neg_case_dict = dict()

check_line_count = 0   
#data 5[2016June] for model #18
#data 16[2016May] for model #13
#data 12[2016Jan] for model #20
#data 11[2015Dec] for model #3
#data 9[2015Oct] for model #3
#----------------------------------
#try data 12[2016Jan] for model #5
#----------------------------------

preds = np.zeros((929615,1))
#for model_index_major in [3,5,13]:
for model_index_major in range(1,25):
    
    train_data_path = './train/model_hierarchy/cust_product_'+str(model_index_major)+'.train'

    dtrain = xgb.DMatrix(train_data_path)

    # specify parameters via map
    param ={"objective" : "multi:softprob",    # multiclass classification 
                  "num_class" : 2,    # number of classes 
                  "eval_metric" : "mlogloss",    # evaluation metric 
                  "nthread" : 6,   # number of threads to be used 
                  "max_depth" : 14,    # maximum depth of tree 
                  "eta" : 0.03,    # step size shrinkage 
                  "subsample" : 0.7,    # part of data instances to grow tree 
                  "colsample_bytree" : 0.7  # subsample ratio of columns when constructing each tree 
           }

    num_round = 50
    bst = xgb.train(param, dtrain, num_round)
    # make prediction
    
    preds_temp = bst.predict(dtest)
    
    preds = np.c_[preds,preds_temp[:,1]]
    
    print 'model#' +str(model_index_major)+" positive rate:%.2f"%(np.mean(preds_temp[:,1]))

    
    

model#1 positive rate:0.11
model#2 positive rate:0.11
model#3 positive rate:0.31
model#4 positive rate:0.11
model#5 positive rate:0.20
model#6 positive rate:0.11
model#7 positive rate:0.11
model#8 positive rate:0.12
model#9 positive rate:0.11
model#10 positive rate:0.11
model#11 positive rate:0.11
model#12 positive rate:0.14
model#13 positive rate:0.27
model#14 positive rate:0.11
model#15 positive rate:0.11
model#16 positive rate:0.11
model#17 positive rate:0.11
model#18 positive rate:0.29
model#19 positive rate:0.17
model#20 positive rate:0.15
model#21 positive rate:0.11
model#22 positive rate:0.19
model#23 positive rate:0.19
model#24 positive rate:0.38


In [31]:
print preds.shape

(929615, 25)


In [36]:
#hierarchy assign the labels for #3 and #24

for i in [3,24]:
    
    for j in range(0,preds.shape[0]):
       
        
       if i == 3: 
          if preds[j,i] > 0.49:
             preds[j,i] = 2.0 
       if i == 24:
          if preds[j,i] > 0.49:
             preds[j,i] = 1.5 
        
            
            
    

In [37]:
preds_sort = np.argsort(preds, axis=1)
print preds_sort.shape
preds_top7 = np.fliplr(preds_sort)[:,:15]
print preds_top7.shape

label_submit_dict = dict()
for i in range(0,preds_sort.shape[0]):
#for i in range(0,100):
    
    top_product = preds_top7[i,0]
    
    if top_product not in label_submit_dict:
       label_submit_dict[top_product]  = 1
    else:
       label_submit_dict[top_product]  += 1

    

(929615, 25)
(929615, 15)


In [38]:
print 'submission distribution'

for item in label_submit_dict:
    
    print 'label #' + str(item) + ' :%.2f'%(label_submit_dict[item]/float(preds_sort.shape[0])*100.0) 
    #print 'label #' + str(item)


submission distribution
label #3 :32.85
label #4 :0.00
label #5 :1.30
label #6 :0.72
label #7 :0.03
label #8 :0.08
label #9 :0.02
label #10 :0.00
label #12 :0.14
label #13 :5.80
label #14 :0.01
label #16 :0.00
label #17 :0.00
label #18 :9.30
label #19 :2.94
label #20 :0.44
label #21 :0.00
label #22 :1.43
label #23 :2.76
label #24 :42.17


In [39]:
Eout_write = open('./output/TheLastFourRecord_pred_Dec16_xgb.eout', 'w')  

feat_data_path = workspace_path + '/input/test_ver2.csv'

test_read = open(feat_data_path, 'r')
headline = test_read.readline().strip()

line_count = 0
for line in test_read:
    
    line = line.strip().split(',')
    cust_id = int(line[1].replace(' ',''))
    
    prt_line_temp = ''
    
    for i in range(0,13):
        
        product_index = preds_top7[line_count,i]
        
        prt_line_temp += label_dict[product_index] + ' '
        
    prt_line_temp = prt_line_temp[:-1]    
    
    
    prt_line = str(cust_id) + '|' + prt_line_temp + '\n'
    
    
    line_count += 1
    
    Eout_write.write(prt_line)
   
print line_count 
Eout_write.close()
    

929615


In [12]:
if False:   
    Eout_write = open('./output/TheLastFourRecord_pred_xgb_Dec15_m18.eout', 'w')  

    feat_data_path = workspace_path + '/input/test_ver2.csv'

    test_read = open(feat_data_path, 'r')
    headline = test_read.readline().strip()

    line_count = 0
    for line in test_read:

        line = line.strip().split(',')
        cust_id = int(line[1].replace(' ',''))

        prt_line_temp = ''

        for i in range(0,1):

            product_index = preds_top7[line_count,i]

            #test #3 and #24 aauracy..

            if product_index == 18:

               prt_line_temp += label_dict[product_index] + ' '

        if len(prt_line_temp) > 0:
           prt_line_temp = prt_line_temp[:-1]    


        prt_line = str(cust_id) + '|' + prt_line_temp + '\n'


        line_count += 1

        Eout_write.write(prt_line)

    print line_count 
    Eout_write.close()
        

929615


In [28]:
dict_read = open(workspace_path + '/dict/cust_last_prods.txt', 'r')

cust_end_products_dict = dict()

line_count = 0
for line in dict_read:  
    line = line.strip()
    
    line_count += 1
    
    cust_id = int(line.split('|')[0])
    last_prod_date = line.split('|')[1]
    label_tokens_get = line.split('|')[2]

    
    if cust_id not in cust_end_products_dict:
       cust_end_products_dict[cust_id] = [] 
    
    if len(label_tokens_get) > 0 and last_prod_date == '2016-05': 
    
        for feat_item in label_tokens_get.split(','):
            
            prod_id = int(feat_item)
            cust_end_products_dict[cust_id].append(label_dict[prod_id])
        
print line_count 
dict_read.close()

956645


In [40]:
Eout_dict_read = open('./output/TheLastFourRecord_pred_Dec16_xgb.eout', 'r')
#Eout_dict_read = open('./output/TheLastFourRecord_pred_xgb_Dec15_m18.eout', 'r')

id_2_prod_dict = dict()

for line in Eout_dict_read:
    line = line.strip()
    
    if len(line.split('|')) != 2:
       print 'error_line:'+line
    else:
       cust_id  = int(line.split('|')[0])
       products = line.split('|')[1]
        
       id_2_prod_dict[cust_id] = products 

    
sample_submission_read = open('./output/sample_submission.csv', 'r')

head_line = 'ncodpers,added_products' + '\n'
submission_write = open('./output/submission_Dec16-2.csv', 'w')  
submission_write.write(head_line)

check_line_count = 0

duplicated_items_count = 0
#for i in range(0,10000):    
    #line = sample_submission_read.readline().strip()
    
for line in sample_submission_read:
    line = line.strip()    
    
    if 'ncodpers' not in line:
    
        cust_id  = int(line.split(',')[0])

        prt_line = str(cust_id) + ','
        
        pruned_line = False
        
        added_items_count = 0

        if cust_id in id_2_prod_dict:
            
           
           #print cust_end_products_dict[cust_id]
           #print 'predict:'+ str(id_2_prod_dict[cust_id])  
           
           for prod_name_item in id_2_prod_dict[cust_id].split(' '):
            
               
               if cust_id in cust_end_products_dict:
                   if prod_name_item not in cust_end_products_dict[cust_id]:
                        
                         if added_items_count < 7:
                            
                            prt_line += prod_name_item + ' '
                        
                            added_items_count += 1

                            pruned_line = True

                   else: 

                     #print '->' + prod_name_item    

                        duplicated_items_count += 1

        check_line_count += 1
        
        if pruned_line:
           submission_write.write(prt_line[:-1]+'\n')
        else:
           submission_write.write(prt_line+'\n')  

print check_line_count 
print duplicated_items_count
    
submission_write.close() 
sample_submission_read.close() 
Eout_dict_read.close() 

929615
411295
